<a href="https://colab.research.google.com/github/mperetto/Python-Chat-with-audio/blob/main/Chat_with_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat with audio interviews
This ongoing project aims to develop a chat app that allows users to upload an audio file and view the chat history of the file in the format of 'speaker_1: ... speaker_2: ...,' with the ability to ask questions about the interview.

## Project Structure
The project is subdivided in 5 main parts.

1.   Upload and conversion to .wav format of the audio file (using ffmpeg) in case it is not already in .wav format.
2.   Processing the diarization of the audio file using the pyannote model to recognize the speakers in the audio.
3.   Extract the text from the audio with Whisper model.
4.   Matching the audio text segment get by Whisper with the timing of the diarization.
5.   Visualize the extracted text in chat format and permit the Q&A.



### Installing the dependecies

In [ ]:
!apt update && apt install ffmpeg
!pip3 install -q git+https://github.com/linto-ai/whisper-timestamped
!pip install -q https://github.com/pyannote/pyannote-audio/archive/refs/heads/develop.zip

### Import and set up the pyannote pipeline

In [ ]:
from pyannote.audio import Pipeline
import torch

pipeline = Pipeline.from_pretrained(
    'pyannote/speaker-diarization@2.1',
    use_auth_token='YOUR_TOKEN' # To get the token execute this cell and follow the instructions
)
pipeline.to(torch.device('cuda'))

### Import audio file and convert it if necessary

In [ ]:
import subprocess

path = "audio.m4a"

if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

### Compute the diarization

In [ ]:
diarization = pipeline(path, num_speakers=2)

In [ ]:
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"[{turn.start:.1f}, {turn.end:.1f}, {speaker}]")

### Import Whisper model and extract the text

In [ ]:
import whisper_timestamped as whisper

audio = whisper.load_audio(path)

model = whisper.load_model('medium')

result = whisper.transcribe(model, audio, vad=True)

In [ ]:
result["text"]

In [ ]:
words_timestamp = []
for segment in result["segments"]:
  words_timestamp.extend(segment["words"])

In [ ]:
words_timestamp

In [ ]:
len(diarization)

In [ ]:
prev_speaker = ""
chat_history = []
map_speakers = {"SPEAKER_00": "speaker_0", "SPEAKER_01": "speaker_1"}
cont = 0
for turn, _, speaker in diarization.itertracks(yield_label=True):
    cont += 1
    if prev_speaker == "":
      chat_history.append({"role": map_speakers[speaker], "message": ""})
      prev_speaker = speaker
    elif prev_speaker != speaker:
      chat_history.append({"role": map_speakers[speaker], "message": ""})
      prev_speaker = speaker

    for word in words_timestamp:
      if word["start"] >= round(turn.start, 1) and word["end"] <= round(turn.end):
        chat_history[-1]["message"] += f" {word['text']}"

In [ ]:
cont

In [ ]:
chat_history